In [133]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [134]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618


In [135]:
df = dataPrep(ings, prods, prod_ing)
#df.to_csv('recommendations/recommender_products.csv')

In [136]:
df = df[df['ingCount'] > 1]  
#df_cos = df
print('Number of products: ', len(df))

Number of products:  15882


In [126]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

#remove water
df['ing#List'] = [remove_values_from_list(i,0) for i in df['ing#List']]

In [127]:
df.sort_values(by='ingCount', ascending=True).head(3)

,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,...,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount,id2
13891,"[Rosa Canina Fruit Oil, Prunus Amygdalus Dulci...","[713, 139]",100% Natural Rose Hip Oil Serum,Olivia Care,NaN,0.0,0,0,1,0,...,0,0,0,0,1,0,1,0,2,17476
12070,"[Panax Ginseng Extract, Hyaluronic Acid]","[950, 604]",Wild Ginseng Ampoule,Eclado,NaN,0.0,0,0,1,0,...,0,0,0,0,1,1,1,0,2,14490
12080,"[Panthenol, Carica Papaya (Papaya) Fruit Extract]","[26, 223]",Professional Formula Vitamin Ampoule,Eclado,NaN,0.0,0,0,1,0,...,0,0,0,0,1,1,1,0,2,14501


# GET RECOMMENDATIONS


In [128]:
#id: 4350    
product ='Glow Tonic'
ingTest = df[df['product'].str.match(product)]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Water', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Leaf Extract', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Glycolic Acid', 'Ammonium Glycolate', 'Glycerin', 'Butylene Glycol', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Phenoxyethanol', 'Sodium Benzoate', 'Biotin', 'Polysorbate 20', 'Fragrance']
['Water', 'Glycolic Acid', 'Butylene Glycol', 'Glycerin', 'Sodium Hydroxide', 'Aloe Barbadensis Leaf Juice', 'Hamamelis Virginiana (Witch Hazel) Water', 'Aesculus Hippocastanum (Horse Chestnut) Seed Extract', 'Hexylene Glycol', 'Fructose', 'Glucose', 'Sucrose', 'Urea', 'Dextrin', 'Alanine', 'Glutamic Acid', 'Aspartic Acid', 'Hexyl Nicotinate', 'Panax Ginseng Root Extract', 'Ethylhexylglycerin', 'Disodium EDTA', 'Biotin', 'Panthenol', 'PPG-26-Buteth-26', 'PEG-40 Hydrogenated Castor Oil(Perfuming, Surfactant, Fragrance, Emulsifying, 

## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [116]:
%%time

recommended = get_COS_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/COS.csv')

Wall time: 23.5 s


In [117]:
topTen

,product,brand,ingList,commonIng
14768,Glow Tonic To-Go Pads,Pixi,"[Water, Glycolic Acid, Butylene Glycol, Glycer...",20
8032,Revitalising Ginseng Toner With Glycolic Acid 5%,Bravura London,"[Water, Aloe Barbadensis (Aloe Vera) Gel, Hama...",17
13600,Tensage Eye Contour Cream,Biopelle,"[Water, Snail Secretion Filtrate, C12-20 Acid ...",14
3350,Juniper Oil-Free Face Moisturizer,TheBalm,"[Water, Ethylhexyl Methoxycinnamate, Butylene ...",14
812,Glycolic Acid 7% Toning Solution,The Ordinary,"[Water, Glycolic Acid, Rosa Damascena Flower W...",16
9098,Tensage Radiance Eye Cream,Biobelle,"[Water, Snail Secretion Filtrate, C12-20 Acid ...",14
2866,Ultra Facial Oil-Free Lotion,Kiehl's,"[Water, Glycerin, Alcohol Denat., Isononyl Iso...",13
10534,Rehydrating Toning Essence,Su-Man Skincare,"[Water, Rosa Centifolia Flower Water, Hyaluron...",13
2962,Cellular 3-Minute Peel,La Prairie,"[Water, Hydroxyethyl Acrylate/Sodium Acryloyld...",15
1760,Skin Hydrating Masque,Dermalogica,"[Water, Glycerin, Butylene Glycol, Squalane, D...",14


In [92]:
l1 = df.iloc[4350]['ing#List']
l2 = df.iloc[449]['ing#List']

## Average Overlap

In [137]:
#df['ing#List'] = [ast.literal_eval(i) for i in df['ing#List']]
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
idx = indices[product]

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [average_overlap(itemLookup,i) for i in items] ##change method here
sim_scores = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores = sim_scores[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores]

recommended = df[['product', 'brand','ingList']].iloc[product_indices]

topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/AO.csv')

In [138]:
topTen

,product,brand,ingList,commonIng
829,Thermal Spring Water,Avene,"[Water, Nitrogen]",1
4666,Lathering Tube Soap,Bobbi Brown,"[Water, nan]",1
6210,Thermal Spa Water,Vichy,"[Water, Carbon Dioxide]",1
7536,3 to Go Thermal Spring Water Set,Avene,"[Water, Nitrogen]",1
13786,Roman Chamomile Flower Waters,Starwest,"[Water, Anthemis Nobilis Flower Oil]",1
15810,Mineral Spray Natural Mineral Water Facial Spray,Evian,"[Water, Nitrogen]",1
14768,Glow Tonic To-Go Pads,Pixi,"[Water, Glycolic Acid, Butylene Glycol, Glycer...",20
1622,Aloe Vera Toner,Mario Badescu,"[Water, Algae Extract, Methylparaben]",1
2051,Serozinc,La Roche-Posay,"[Water, Sodium Chloride, Zinc Sulfate]",1
3076,Acerola 100,DHC,"[Water, Butylene Glycol, Malpighia Glabra (Ace...",2


## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [139]:
#%%time

recommended = get_RBO_recommendations(df, product, rbo_min)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/RBO.csv')

In [140]:
topTen

,product,brand,ingList,commonIng
14768,Glow Tonic To-Go Pads,Pixi,"[Water, Glycolic Acid, Butylene Glycol, Glycer...",20
3743,Geranium Face Toner,TheBalm,"[Water, Aloe Barbadensis Leaf Juice, Glycolic ...",8
3707,Alcohol-Free Face Toner,TheBalm,"[Water, Hamamelis Virginiana (Witch Hazel) Lea...",9
8285,Clear Skin Hydrator Gel,Alpha-H,"[Water, Hamamelis Virginiana (Witch Hazel) Lea...",6
694,Seaweed Cleansing Lotion,Mario Badescu,"[Water, Aloe Barbadensis Leaf Juice, Propylene...",7
7541,Sensitive Facial Cleansing Towelettes with Cot...,Burt's Bees,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",8
1372,Facial Radiance Pads,First Aid Beauty,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",5
11875,Orchid Flower Aqua UV Sun Gel,The Orchid Skin,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",6
10629,Green Cover Sunscreen SPF50+ PA+++,Cosmo Monster,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",5
6271,Sensitive Facial Cleansing Towelettes,Burt's Bees,"[Water, Aloe Barbadensis Leaf Juice, Glycerin,...",8
